In [1]:
import torch
from pytorch_transformers import BertTokenizer, BertForMaskedLM

In [29]:
S1 = '[CLS] the company has a fiduciary duty to its shareholders . [SEP]'
S2 = 'one of its many regulatory requirements . [SEP]'

S1 = '[CLS] the volatility caused us to what [SEP]'
S2 = 'one of its many regulatory requirements . [SEP]'

tokenizer      = BertTokenizer.from_pretrained('bert-base-uncased')
text           = f'{S1} {S2}'
tokenized_text = tokenizer.tokenize(text)

for i, word in enumerate(tokenized_text):
    print("{} {}".format(i, word))
    
print(tokenized_text)


0 [CLS]
1 the
2 vol
3 ##ati
4 ##lity
5 caused
6 us
7 to
8 what
9 [SEP]
10 one
11 of
12 its
13 many
14 regulatory
15 requirements
16 .
17 [SEP]
['[CLS]', 'the', 'vol', '##ati', '##lity', 'caused', 'us', 'to', 'what', '[SEP]', 'one', 'of', 'its', 'many', 'regulatory', 'requirements', '.', '[SEP]']


In [30]:
MI = [8,12]

for i in MI :
    tokenized_text[i] = '[MASK]'

print(tokenized_text)

indexed_tokens   = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids     = [0] * len(tokenizer.tokenize(S1)) + [1] * len(tokenizer.tokenize(S2))

tokens_tensor    = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

model            = {}
model['GooBERT'] = BertForMaskedLM.from_pretrained('GooBERT')
model['FinBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Prime_128MSL-250K')
model['PreBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Pre2K_128MSL-250K')
model['ComBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Combo_128MSL-250K')

preds  = {}
for m in model:
    with torch.no_grad():
        preds[m] = model[m](tokens_tensor, token_type_ids = segments_tensors)[0]
        
for m in preds:
    tokens = []
    for i in MI:
        predicted_index = torch.argmax(preds[m][0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        tokens.append(predicted_token)

    print(f'{m} : {tokens}')

['[CLS]', 'the', 'vol', '##ati', '##lity', 'caused', 'us', 'to', '[MASK]', '[SEP]', 'one', 'of', '[MASK]', 'many', 'regulatory', 'requirements', '.', '[SEP]']
GooBERT : ['.', 'our']
FinBERT : [':', 'the']
PreBERT : ['31', 'the']
ComBERT : [':', 'the']
